In [1]:
import torch
import math
import matplotlib.pyplot as plt

# Physical parameters
g = 9.81         # gravitational acceleration
l = 1.0          # pendulum length
k_damp = 0.05    # damping coefficient

def pendulum_acceleration(x, v, t, f_drive, omega):
    """
    Computes the acceleration of the pendulum given:
      x: angular displacement (tensor),
      v: angular velocity (tensor),
      t: current time (float),
      f_drive: driving amplitude (float),
      omega: driving frequency (float).
      
    The force is applied as a square wave using sign(sin(omega*t)).
    
    Note: To ensure torch.sin receives a Tensor instead of a float,
    we wrap the scalar omega*t in torch.tensor().
    """
    # Convert omega*t to a torch tensor so that torch.sin can work correctly.
    sine_arg = torch.tensor(omega * t, dtype=torch.float)
    drive = f_drive * torch.sign(torch.sin(sine_arg))
    a = -(g / l) * torch.sin(x) - k_damp * v + drive
    return a

def rk4_step(x, v, t, dt, f_drive, omega):
    """
    Advances the state (x, v) by one Runge–Kutta 4 step.
    """
    # k1
    a1 = pendulum_acceleration(x, v, t, f_drive, omega)
    k1_x = v
    k1_v = a1

    # k2
    x2 = x + 0.5 * dt * k1_x
    v2 = v + 0.5 * dt * k1_v
    a2 = pendulum_acceleration(x2, v2, t + 0.5 * dt, f_drive, omega)
    k2_x = v2
    k2_v = a2

    # k3
    x3 = x + 0.5 * dt * k2_x
    v3 = v + 0.5 * dt * k2_v
    a3 = pendulum_acceleration(x3, v3, t + 0.5 * dt, f_drive, omega)
    k3_x = v3
    k3_v = a3

    # k4
    x4 = x + dt * k3_x
    v4 = v + dt * k3_v
    a4 = pendulum_acceleration(x4, v4, t + dt, f_drive, omega)
    k4_x = v4
    k4_v = a4

    # Combine increments
    new_x = x + (dt / 6.0) * (k1_x + 2 * k2_x + 2 * k3_x + k4_x)
    new_v = v + (dt / 6.0) * (k1_v + 2 * k2_v + 2 * k3_v + k4_v)
    return new_x, new_v

def simulate_pendulum(f_drive, omega, num_cycles=5, samples_per_cycle=1000):
    """
    Simulates the forced pendulum for a given number of drive cycles.
    Each simulation starts from initial conditions x=0, v=0.
    
    Parameters:
      f_drive: driving amplitude (float)
      omega: driving frequency (float)
      num_cycles: number of cycles in the simulation
      samples_per_cycle: number of reservoir samples per cycle
      
    Returns:
      A 1D torch tensor containing the transient dynamics (the reservoir state).
    """
    period = 2 * math.pi / omega
    T = num_cycles * period
    total_steps = num_cycles * samples_per_cycle
    dt = T / total_steps  # time step

    # initial conditions: x = 0, v = 0
    x = torch.tensor(0.0)
    v = torch.tensor(0.0)
    t = 0.0

    x_traj = []
    for i in range(total_steps):
        x, v = rk4_step(x, v, t, dt, f_drive, omega)
        t += dt
        x_traj.append(x.item())
    return torch.tensor(x_traj, dtype=torch.float)

def target_polynomial(x):
    """
    Computes the seventh-degree polynomial:
      f(x) = (x-3)(x-2)(x-1)x(x+1)(x+2)(x+3)
    """
    return (x - 3) * (x - 2) * (x - 1) * x * (x + 1) * (x + 2) * (x + 3)

def build_dataset(num_samples, num_cycles=5, samples_per_cycle=1000, encoding='frequency'):
    """
    Constructs a dataset for Task I.
    
    For amplitude encoding, each input u (sampled uniformly from [-3,3])
    is encoded by setting the driving amplitude as:
      f_drive = 1 + (u + 3) / 6.
    The target is the polynomial value at u.
    
    Returns:
      u: original input values (shape: [num_samples])
      reservoir_matrix: reservoir states (shape: [num_samples, reservoir_dimension])
      y_target: target polynomial values (shape: [num_samples])
    """
    # Generate inputs u in [-3, 3]
    u = 6 * torch.rand(num_samples) - 3
    y_target = target_polynomial(u)
    
    if encoding == 'amplitude':
        # Map input u to driving amplitude f_drive in [1, 2]
        f_drive_vals = 1 + (u + 3) / 6
        omega_vals = torch.ones_like(u)  # constant driving frequency (e.g., 1.0)
    elif encoding == 'frequency':
        # Alternatively, for frequency encoding (not used in this example)
        f_drive_vals = torch.ones_like(u)
        omega_vals = 0.5 + (u + 3) / 6  # example: omega in [0.5, 1.5]
    else:
        raise ValueError("Encoding must be 'amplitude' or 'frequency'")
    
    reservoir_states = []
    for i in range(num_samples):
        # Convert the encoded values to native Python numbers.
        f_drive = f_drive_vals[i].item()
        omega = omega_vals[i].item()
        x_traj = simulate_pendulum(f_drive, omega, num_cycles, samples_per_cycle)
        reservoir_states.append(x_traj)
    reservoir_matrix = torch.stack(reservoir_states, dim=0)
    return u, reservoir_matrix, y_target

class LinearReadout(torch.nn.Module):
    def __init__(self, input_dim):
        super(LinearReadout, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return self.linear(x)

def train_linear_readout(train_features, train_targets, num_epochs=500, learning_rate=1e-3):
    num_samples, input_dim = train_features.shape
    model = LinearReadout(input_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    train_targets = train_targets.view(-1, 1)
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(train_features)
        loss = criterion(outputs, train_targets)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")
    return model

def main():
    torch.manual_seed(0)  

    num_train = 500
    u_train, train_features, y_train = build_dataset(num_train, num_cycles=10, samples_per_cycle=100, encoding='amplitude')
    print("Training dataset built. Reservoir state dimension:", train_features.shape[1])
    
    model = train_linear_readout(train_features, y_train, num_epochs=500, learning_rate=1e-3)
    
    num_test = 100
    u_test, test_features, y_test = build_dataset(num_test, num_cycles=5, samples_per_cycle=100, encoding='amplitude')
    
    # Evaluate the trained model on the test set using MSE loss
    with torch.no_grad():
        predictions = model(test_features).squeeze()
        test_loss = torch.mean((predictions - y_test) ** 2).item()
    print(f"Test Loss: {test_loss:.6f}")
    
    plt.figure()
    plt.scatter(u_test.numpy(), y_test.numpy(), label="Target", color="blue")
    plt.scatter(u_test.numpy(), predictions.numpy(), label="Prediction", color="red")
    plt.xlabel("Input u (in [-3, 3])")
    plt.ylabel("Polynomial Value")
    plt.legend()
    plt.title("Reservoir Computing Performance on Polynomial Approximation")
    plt.show()

if __name__ == "__main__":
    main()


Training dataset built. Reservoir state dimension: 1000
Epoch 50/500, Loss: 1225.786621
Epoch 100/500, Loss: 1030.076294
Epoch 150/500, Loss: 888.078125
Epoch 200/500, Loss: 779.193848
Epoch 250/500, Loss: 695.286133
Epoch 300/500, Loss: 631.187927
Epoch 350/500, Loss: 582.851318
Epoch 400/500, Loss: 546.890869
Epoch 450/500, Loss: 520.446289
Epoch 500/500, Loss: 501.133423


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x500 and 1000x1)